# CS7641 Machine Learning
*Application of Machine Learning in Pairs Trading*

In [244]:
import pandas as pd
import numpy as np
import os
import datetime
import math
import sklearn
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Price History Table
Here is the price table we used for this function. I used the top 3 stocks as an example. I'll change the data after I get any pair.

In [245]:
# Import training dataset
training_set = pd.read_csv("training_data.csv")
training_set.head(3)

,PERMNO,price_20070103,price_20070104,price_20070105,price_20070108,price_20070109,price_20070110,price_20070111,price_20070112,price_20070116,price_20070117,price_20070118,price_20070119,price_20070122,price_20070123,price_20070124,price_20070125,price_20070126,price_20070129,price_20070130,price_20070131,price_20070201,price_20070202,price_20070205,price_20070206,price_20070207,price_20070208,price_20070209,price_20070212,price_20070213,price_20070214,price_20070215,price_20070216,price_20070220,price_20070221,price_20070222,price_20070223,price_20070226,price_20070227,price_20070228,...,at_turn_2012-09,at_turn_2012-10,at_turn_2012-11,at_turn_2012-12,at_turn_2013-01,at_turn_2013-02,at_turn_2013-03,at_turn_2013-04,at_turn_2013-05,at_turn_2013-06,at_turn_2013-07,at_turn_2013-08,at_turn_2013-09,at_turn_2013-10,at_turn_2013-11,at_turn_2013-12,at_turn_2014-01,at_turn_2014-02,at_turn_2014-03,at_turn_2014-04,at_turn_2014-05,at_turn_2014-06,at_turn_2014-07,at_turn_2014-08,at_turn_2014-09,at_turn_2014-10,at_turn_2014-11,at_turn_2014-12,at_turn_2015-01,at_turn_2015-02,at_turn_2015-03,at_turn_2015-04,at_turn_2015-05,at_turn_2015-06,at_turn_2015-07,at_turn_2015-08,at_turn_2015-09,at_turn_2015-10,at_turn_2015-11,at_turn_2015-12
0,10025,49.06,50.34,49.63,49.50,50.62,49.68,49.20,48.90,46.83,47.05,46.32,47.49,45.11,44.85,44.57,45.90,45.55,45.80,46.01,46.63,45.50,46.07,45.41,44.48,45.71,46.90,45.71,46.795,46.91,46.27,46.49,46.66,48.05,47.89,47.24,46.25,45.75,44.00,45.32,...,2.685,2.685,2.685,2.721,2.721,2.721,2.697,2.697,2.697,2.614,2.614,2.614,2.548,2.548,2.548,2.533,2.533,2.533,2.490,2.490,2.490,2.485,2.485,2.485,2.501,2.501,2.501,2.598,2.598,2.598,2.628,2.628,2.628,2.674,2.674,2.674,2.730,2.730,2.730,2.573
1,10026,41.02,41.89,39.66,40.42,40.54,41.54,41.41,42.23,41.71,41.28,42.74,42.76,42.53,41.85,42.20,41.94,41.53,41.64,41.67,41.28,41.57,40.98,41.20,41.35,41.48,41.32,40.93,41.130,41.46,41.41,41.25,41.77,41.46,40.59,41.44,40.41,40.01,37.47,39.69,...,1.432,1.432,1.440,1.440,1.440,1.434,1.434,1.434,1.422,1.422,1.422,1.413,1.413,1.413,1.390,1.390,1.390,1.372,1.372,1.372,1.354,1.354,1.354,1.355,1.355,1.355,1.362,1.362,1.362,1.340,1.340,1.340,1.348,1.348,1.348,1.357,1.357,1.357,1.349,1.349
2,10032,24.12,24.30,23.89,23.70,23.13,20.90,20.84,21.58,20.79,20.58,19.97,20.21,20.05,20.61,21.22,18.18,17.62,17.35,16.86,16.80,16.86,17.29,17.22,17.22,17.68,17.71,17.57,17.610,17.39,16.85,17.01,17.00,17.45,17.23,17.21,17.20,17.31,16.65,16.40,...,1.671,1.671,1.701,1.701,1.701,1.651,1.651,1.651,1.616,1.616,1.616,1.580,1.580,1.580,1.560,1.560,1.560,1.532,1.532,1.532,1.523,1.523,1.523,1.519,1.519,1.519,1.556,1.556,1.556,1.604,1.604,1.604,1.623,1.623,1.623,1.627,1.627,1.627,1.603,1.603


In [246]:
# Filtering the table only for the price history
filter_col = [col for col in training_set if col.startswith('price_')]
training_set_price = training_set[filter_col]
training_set_price.head(3)

,price_20070103,price_20070104,price_20070105,price_20070108,price_20070109,price_20070110,price_20070111,price_20070112,price_20070116,price_20070117,price_20070118,price_20070119,price_20070122,price_20070123,price_20070124,price_20070125,price_20070126,price_20070129,price_20070130,price_20070131,price_20070201,price_20070202,price_20070205,price_20070206,price_20070207,price_20070208,price_20070209,price_20070212,price_20070213,price_20070214,price_20070215,price_20070216,price_20070220,price_20070221,price_20070222,price_20070223,price_20070226,price_20070227,price_20070228,price_20070301,...,price_20151104,price_20151105,price_20151106,price_20151109,price_20151110,price_20151111,price_20151112,price_20151113,price_20151116,price_20151117,price_20151118,price_20151119,price_20151120,price_20151123,price_20151124,price_20151125,price_20151127,price_20151130,price_20151201,price_20151202,price_20151203,price_20151204,price_20151207,price_20151208,price_20151209,price_20151210,price_20151211,price_20151214,price_20151215,price_20151216,price_20151217,price_20151218,price_20151221,price_20151222,price_20151223,price_20151224,price_20151228,price_20151229,price_20151230,price_20151231
0,49.06,50.34,49.63,49.50,50.62,49.68,49.20,48.90,46.83,47.05,46.32,47.49,45.11,44.85,44.57,45.90,45.55,45.80,46.01,46.63,45.50,46.07,45.41,44.48,45.71,46.90,45.71,46.795,46.91,46.27,46.49,46.66,48.05,47.89,47.24,46.25,45.75,44.00,45.32,44.14,...,80.00,79.35,86.34,87.47,86.88,86.72,83.00,86.51,85.50,83.54,84.92,84.60,85.46,87.43,89.66,88.16,89.03,91.18,89.35,90.68,89.89,93.29,88.53,86.75,86.84,83.60,83.92,80.51,82.61,82.74,80.80,79.01,76.40,77.93,78.98,80.55,78.97,77.15,77.22,77.15
1,41.02,41.89,39.66,40.42,40.54,41.54,41.41,42.23,41.71,41.28,42.74,42.76,42.53,41.85,42.20,41.94,41.53,41.64,41.67,41.28,41.57,40.98,41.20,41.35,41.48,41.32,40.93,41.130,41.46,41.41,41.25,41.77,41.46,40.59,41.44,40.41,40.01,37.47,39.69,38.84,...,123.95,123.48,114.87,113.83,113.73,114.99,113.93,113.73,114.71,113.99,114.98,115.18,115.65,116.14,116.46,117.47,118.06,116.68,117.15,115.71,115.02,116.50,116.37,117.04,115.58,114.86,114.31,114.70,115.35,115.69,114.37,112.69,112.92,114.53,115.90,118.21,117.64,119.37,119.25,116.67
2,24.12,24.30,23.89,23.70,23.13,20.90,20.84,21.58,20.79,20.58,19.97,20.21,20.05,20.61,21.22,18.18,17.62,17.35,16.86,16.80,16.86,17.29,17.22,17.22,17.68,17.71,17.57,17.610,17.39,16.85,17.01,17.00,17.45,17.23,17.21,17.20,17.31,16.65,16.40,16.56,...,35.87,36.92,37.17,36.58,36.65,37.17,36.79,35.77,36.46,36.11,36.41,36.94,37.22,37.07,37.10,37.34,37.96,37.20,37.78,37.63,37.40,37.51,36.96,36.49,35.48,35.50,34.94,34.66,35.32,35.57,35.26,34.99,35.22,35.08,35.95,36.05,35.67,35.92,35.46,34.92


## Create the spread funtion (price pair's relation)
We will create the function of spread in here. The basic function of spread is defined as blow:

$Spread = log(a) - nlog(b)$

where the 'a' and 'b' are prices of stocks A and B respectively and the 'n' is hedge ratio. Our target is finding dynamics of spread based on the machine learning. We will use the supervised machine learning to implement this part and the possible candidates are 'linear regression' and 'support vector machine (SVM)'

In [249]:
def create_spread_function(a, b, start_t, end_t, alg='log'):
    """
    * Input
        - a, b: Stock A and B's price history
        - start_t, end_t: start/end time of the analysis on the data. 
            They use the same unit with the data. For example, 0 means the 
            first data of the a and b. 
            (Analyze the data from a[start_t], b[start_t] to a[end_t], 
            b[end_t])
        - alg: Type of algorithm. The 'log' means the log normalization
    * Output
        - The function of spread. Output of this function is spread and z_score.
    * Function
        - Apply the supervised machine learning to find the dynamics of
        spread
    """
    
    def log_spread_func(a, b):
        spread = math.log(a) - w_avg * math.log(b)
        z_score = spread/w_std

        return (spread, z_score)

    def lr_spread_func(a, b):
        spread = a - w_avg * b
        z_score = spread/w_std

        return (spread, z_score)

    def svm_spread_func(a, b):
        pass

    # Slice the date
    target_a = a[start_t:end_t]
    target_b = b[start_t:end_t]
    total_date = end_t-start_t

    # Find the coefficient of the log normalization
    if alg == 'log':
        print("Log normalization")
        # use the log function
        target_a = np.log(target_a)
        target_b = np.log(target_b)

        # Calculate the weight
        w_list = target_b/target_a
        w_avg = np.average(w_list)

        # Calculate the standard deviation for the z-score calculation
        w_std = np.std(w_list)

        return log_spread_func

    # Find the coefficient of the linear regression
    elif alg == 'lr':
        print("Linear regression")
        # Train the data using linear regression
        target_a = target_a[:,np.newaxis]
        regr = linear_model.LinearRegression()
        regr.fit(target_a, target_b)
        
        # Calculate the weight
        predict_b = regr.predict(target_a)
        w_list = target_b/predict_b
        w_avg = np.average(w_list)

        # Calculate the standard deviation for the z-score calculation
        w_std = np.std(w_list)

        return lr_spread_func

    elif alg == 'svm':
        print("Support Vector Machine")
        return svm_spread_func

    print("Check the algorithm. Input was " + alg)

    pass

### How to use

Here, we will see how to use the spread function.
Right now, the results are bad because the stock a and b is randomly choosen and does not have any relation.

In [250]:
# Generate input for the test
a = training_set_price.loc[0]
b = training_set_price.loc[1]

# Check the function based on the log normalization
spread_func = create_spread_function(a, b, 0, 1000, 'log')
(spread, z_score) = spread_func(a[0], b[0])
print(spread, z_score)

# Check the function based on the linear regression
spread_func = create_spread_function(a, b, 0, 1000, 'lr')
(spread, z_score) = spread_func(a[0], b[0])
print(spread, z_score)

Log normalization
-0.1478672531468277 -1.2993837129839456
Linear regression
8.042285297435534 53.31521967302066


## Generate the spread function list
We also generated the spread function list. The spread function list is needed for the backtesting. It will be used in the z-score matrix generation

In [251]:
def gen_spread_func_list(stock_pair_list, training_set, start_t, end_t, alg):
    """
    * Input: stock_pair_list, training_set, start_t, end_t, algorithm
    * Output: spread_func_list
    * Function: generate all the spread function based on the stock pairs list
    """
    spread_func_list = []

    # Let the training_set includes only PERMNO and price history
    filter_col = ['PERMNO']
    price_list = [col for col in training_set if col.startswith('price_')]
    filter_col.extend(price_list)
    training_set = training_set[filter_col]

    for pair in stock_pair_list:
        # Generate the price data for each stock in the pair
        training_set_a = training_set.loc[training_set['PERMNO'] == pair[0]]
        training_set_a = training_set_a[price_list].iloc[0]
        training_set_b = training_set.loc[training_set['PERMNO'] == pair[1]]
        training_set_b = training_set_b[price_list].iloc[0]

        # Run the create_spread_function
        spread_func = create_spread_function(
            training_set_a, b, start_t, end_t, alg)
        spread_func_list.append(spread_func) 

    return spread_func_list

### How to use
We will test the function with the 3 input pairs ([10025, 10026], [10025, 10032], [10026, 10032]). The stocks are distinguished based on the PERMNO. 

In [252]:
# Generate the inputs for the test
stock_pair_list = [[10025, 10026], [10025, 10032], [10026, 10032]]
a = training_set_price.loc[0]
b = training_set_price.loc[1]

# Generate the list of the spread functions
spread_func_list = gen_spread_func_list(
    stock_pair_list, training_set, 0, 1000, 'lr')

# Try to run one of the spread function
(spread, z_score) = spread_func_list[0](a[0], b[0])
print(spread, z_score)


Linear regression
Linear regression
Linear regression
8.042285297435534 53.31521967302066


## Generate the z-score matrix
Generate the z-score matrix based on the spread function list.
It will be used for our strategy to deciding how much stocks we will buy/sell.
* Input: stock pairs list, stock's present price list
* Output: z-score matrix
* Function: calculate the z-scores in the stock pairs list and generate the matrix about z-score

In [253]:
def gen_z_score_matrix(stock_pair_list, current_price_list, spread_func_list):
    """
    * Input: stock_pair_list, current_price_list, spread_func_list
    * Output: z-score matrix
    * Function: run the spread function with the current price list. 
    """

    # Generate the empty data frame based on the stock_pair_list
    columns = []
    index = []
    for pair in stock_pair_list:
        if pair[0] in index:
            pass
        else:
            index.append(pair[0])
        
        if pair[1] in columns:
            pass
        else:
            columns.append(pair[1])
        
    z_score_matrix = pd.DataFrame(index=index, columns = columns)

    # Fill the data in the data frame
    for i, pair in enumerate(stock_pair_list):

        # Find the current price
        price_a = current_price_list[current_price_list['PERMNO']==pair[0]]
        price_a = price_a.iloc[0,1]
        price_b = current_price_list[current_price_list['PERMNO']==pair[1]]
        price_b = price_b.iloc[0,1]

        # Run the spread_function and calculate the z_score
        (spread, z_score) = spread_func_list[i](price_a, price_b)

        # Fill the z_score on the data frame
        z_score_matrix.at[pair[0], pair[1]] = z_score

    return z_score_matrix

### How to use
We will test the function with the 3 input pairs ([10025, 10026], [10025, 10032], [10026, 10032]). The stocks are distinguished based on the PERMNO.

In [254]:
# Generate the input for the test
stock_pair_list = [[10025, 10026], [10025, 10032], [10026, 10032]]
filter_col = ['PERMNO', 'price_20070103']
current_price_list = training_set[filter_col]

# Run the z-score matrix generation function
z_score_matrix = gen_z_score_matrix(
    stock_pair_list, current_price_list, spread_func_list)

print(z_score_matrix)

         10026        10032
10025  53.3152      165.345
10026      NaN  1.20342e+18



## Generate buy/sell the stock matrix
Based on the z-score matrix, we need to decide which stocks to by and which stocks to sell. We will buy the stock if the z-score is above certain point. (Ex. 2-sigma) It will be the argument. In the same way, if the z-score reaches below the certain point, we will sell the stock. (Ex. 0) We also need to care about when the pair of stocks shows the unusual behavior. When we buy at the 2-sigma and if it reaches the 3-sigma, we should sell the stock to stop the loss. 
* Input: entry point, stop loss point, take profit point, commission for trade, z-score matrix
* Output: buy matrix, sell matrix (each component is amount of stock we will buy/sell)
* Function: decide our action based on the z-score matrix

In [ ]:
def gen_action_matrix():
    pass

In [ ]:
# Code verification